In [3]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

drive.mount('/content/drive')


# Define the path to the folder containing the labeled images
train_data_dir ='/content/drive/MyDrive/Dataset_Project/Training_set'
validation_data_dir = '/content/drive/MyDrive/Dataset_Project/Testing_set'

# Define image parameters
img_width, img_height = 416, 416
input_shape = (img_width, img_height, 3)

# Define batch size and number of epochs
batch_size = 16 # à changer
epochs = 10

# Define data augmentation
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Define training and validation generators
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])

# Train the model
model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator))

# Save the model
save_path = '/content/drive/MyDrive/Dataset_Project/CNN_model.h5'
model.save(save_path)



Mounted at /content/drive
Found 423 images belonging to 2 classes.
Found 164 images belonging to 2 classes.


<ipython-input-3-a0c16cde4df1>:65: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
27/27 [==============================] - 197s 7s/step - loss: 0.9762 - accuracy: 0.6454 - val_loss: 0.4297 - val_accuracy: 0.7317
Epoch 2/10
27/27 [==============================] - 170s 6s/step - loss: 0.4881 - accuracy: 0.8582 - val_loss: 0.3776 - val_accuracy: 0.8598
Epoch 3/10
27/27 [==============================] - 166s 6s/step - loss: 0.1320 - accuracy: 0.9385 - val_loss: 0.3572 - val_accuracy: 0.8963
Epoch 4/10
27/27 [==============================] - 166s 6s/step - loss: 0.1132 - accuracy: 0.9622 - val_loss: 0.3658 - val_accuracy: 0.8780
Epoch 5/10
27/27 [==============================] - 167s 6s/step - loss: 0.3017 - accuracy: 0.9574 - val_loss: 0.1088 - val_accuracy: 0.9756
Epoch 6/10
27/27 [==============================] - 174s 6s/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 0.1865 - val_accuracy: 0.9390
Epoch 7/10
27/27 [==============================] - 166s 6s/step - loss: 0.0841 - accuracy: 0.9716 - val_loss: 0.1558 - val_accuracy: 0.9756
Epoch 8/10
27

In [6]:
import numpy as np
from tensorflow.keras.preprocessing import image


# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/Dataset_Project/CNN_model.h5')

# Define the path to the image you want to classify
img_path = '/content/drive/MyDrive/Dataset_Project/Application/oeil_c.jpg'

# Load the image and resize it to match the input size of the model
img = image.load_img(img_path, target_size=(416, 416))

# Convert the image to a numpy array and normalize its values
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0

# Make a prediction using the model and print the result
prediction = model.predict(x)
if prediction[0][0] < 0.5:
    print('No cataract')
else:
    print('Cataract')


1/1 [==============================] - 0s 237ms/step
No cataract
